# DL 5th HW

source: https://medium.com/@utk.is.here/keep-calm-and-train-a-gan-pitfalls-and-tips-on-training-generative-adversarial-networks-edd529764aa9

In [0]:
colab=True

In [2]:
from keras.layers import Input, Dense, Flatten, Dropout, Reshape
from keras.layers import BatchNormalization, Activation, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.optimizers import Adam

from keras.datasets import cifar10
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np

# making sure we used the newest versions
!pip install --upgrade keras 
!pip install --upgrade tensorflow

# for data handling
import pandas as pd
from collections import defaultdict, OrderedDict

# for path handling
from pathlib import Path

# for file handling
import os
import warnings

# Only if working from Google Colab
if colab:
    # Mount the Google Drive to reach its data
    from google.colab import drive
    drive.mount(Path(r'/content/gdrive'))
    # Make the Working Directory where the dataset is
    os.chdir(Path(r'/content/gdrive/My Drive/DL_HW5'))
    # making sure we used the newest versions

# Progressbar generator
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.12.0)
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
     \ 512kB 585kB/s
  Running setup.py bdist_wheel for tqdm ... - \ done
  Stored in directory: /tmp/pip-ephem-wheel-cache-sdug0qob/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
spacy 2.0.18 has requirement numpy>=1.15.0, but you'l

In [0]:
SAVE_DIR = './cifar_10/1'

if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [0]:
def get_generator(input_layer):
    '''
    Requires the input layer as input, outputs the model and the final layer
    '''
    hid = Dense(128 * 16 * 16, activation='relu')(input_layer)    
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)
    hid = Reshape((16, 16, 128))(hid)

    hid = Conv2D(128, kernel_size=5, strides=1,padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2DTranspose(128, 4, strides=2, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2D(128, kernel_size=5, strides=1, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)
                       
    hid = Conv2D(3, kernel_size=5, strides=1, padding="same")(hid)
    out = Activation("tanh")(hid)

    model = Model(input_layer, out)
    model.summary()
  
    return model, out


In [0]:
def get_discriminator(input_layer):
    '''
    Requires the input layer as input, outputs the model and the final layer
    '''

    hid = Conv2D(128, kernel_size=3, strides=1, padding='same')(input_layer)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Conv2D(128, kernel_size=4, strides=2, padding='same')(hid)
    hid = BatchNormalization(momentum=0.9)(hid)
    hid = LeakyReLU(alpha=0.1)(hid)

    hid = Flatten()(hid)
    out = Dense(1, activation='sigmoid')(hid)

    model = Model(input_layer, out)

    model.summary()

    return model, out

In [0]:
from keras.preprocessing import image

def generate_noise(n_samples, noise_dim):
    X = np.random.normal(0, 1, size=(n_samples, noise_dim))
    return X

def show_imgs(epoch, savedir=None):
    noise = generate_noise(9, 100)
    gen_imgs = generator.predict(noise)

    fig, axs = plt.subplots(3, 3)
    count = 0
    for i in range(3):
        for j in range(3):
      # Dont scale the images back, let keras handle it
            img = image.array_to_img(gen_imgs[count], scale=True)
            axs[i,j].imshow(img)
            axs[i,j].axis('off')
            count += 1
    fig.suptitle("Sample after %d epoch" % epoch, fontsize=16)
    
    filename = ("sample_%d.png" % epoch)
    plt.savefig(os.path.join(savedir, filename))
    
    plt.show()
    plt.close('all')

In [7]:
# GAN creation
img_input = Input(shape=(32,32,3))
discriminator, disc_out = get_discriminator(img_input)
discriminator.compile(optimizer=Adam(0.0001, 0.2), loss='binary_crossentropy', metrics=['accuracy'])

discriminator.trainable = False

noise_input = Input(shape=(100,))
generator, gen_out = get_generator(noise_input)

gan_input = Input(shape=(100,))
x = generator(gan_input)
gan_out = discriminator(x)
gan = Model(gan_input, gan_out)
gan.summary()

gan.compile(optimizer=Adam(0.0001, 0.2), loss='binary_crossentropy')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       262272    
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 16, 128)       512       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 16, 16, 128)       0         
__________

In [9]:
BATCH_SIZE = 128

# # Get training images
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Select only Cats
X_train = X_train[y_train[:,0]==3]
X_test = X_test[y_test[:,0]==3]
print ("Training shape: {}".format(X_train.shape))
print ("Testing shape: {}".format(X_test.shape))

# Normalize data
X_train = (X_train - 127.5) / 127.5
X_test = (X_test - 127.5) / 127.5
 
num_batches = int(X_train.shape[0]/BATCH_SIZE)

170500096/170498071 [==============================] - 70s 0us/step
Training shape: (5000, 32, 32, 3)
Testing shape: (1000, 32, 32, 3)


# Train

Main differences between a common DCGAN that:
1. It uses soft labels, meaning instead of using hard labels (1 for generated image and 0 for true) it randomly generates between 0.9 and 1 for generated images and between 0 and 0.1 for true images. Only for the discriminator, because it keeps from rapidly go to 0.

2. Flips 5% of labels to opposite labels . This is for only the discriminator. 


In [12]:
N_EPOCHS = 200
initial_epoch=1
SAVE_INTERVAL=5
logs=dict(d_loss=[],d_true_loss=[], d_gene_loss=[], g_loss=[])
logs_val=dict(d_val_loss=[], d_true_val_loss=[], d_gene_val_loss=[], d_true_val_acc=[], d_gene_val_acc=[])

for epoch in tqdm(range(initial_epoch, N_EPOCHS+1)):
    discriminator_true_loss=[]
    discriminator_gene_loss=[]
    generator_loss=[]
    np.random.shuffle(X_train)
    for batch_idx in tqdm(range(num_batches),desc=('Epoch #%d' % epoch), leave=False):
        # Get the next set of real images to be used in this iteration
        images = X_train[batch_idx*BATCH_SIZE : (batch_idx+1)*BATCH_SIZE]

        noise_data = generate_noise(BATCH_SIZE, 100)
        generated_images = generator.predict(noise_data)

        # Train on soft labels (add noise to labels as well)
        noise_prop = 0.05 # Randomly flip 5% of labels

        # Prepare labels for real data
        true_labels = np.zeros((BATCH_SIZE, 1)) + np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(true_labels)), size=int(noise_prop*len(true_labels)))
        true_labels[flipped_idx] = 1 - true_labels[flipped_idx]

        # Train discriminator on real data
        d_loss_true = discriminator.train_on_batch(images, true_labels)
        discriminator_true_loss.append(d_loss_true)
        
        # Prepare labels for generated data
        gene_labels = np.ones((BATCH_SIZE, 1)) - np.random.uniform(low=0.0, high=0.1, size=(BATCH_SIZE, 1))
        flipped_idx = np.random.choice(np.arange(len(gene_labels)), size=int(noise_prop*len(gene_labels)))
        gene_labels[flipped_idx] = 1 - gene_labels[flipped_idx]

        # Train discriminator on generated data
        d_loss_gene = discriminator.train_on_batch(generated_images, gene_labels)
        discriminator_gene_loss.append(d_loss_gene)

        # Train generator
        noise_data = generate_noise(BATCH_SIZE, 100)
        g_loss = gan.train_on_batch(noise_data, np.zeros((BATCH_SIZE, 1)))
        generator_loss.append(g_loss)
       
    
    discriminator_true_loss=np.array(discriminator_true_loss).mean(axis=0)
    discriminator_gene_loss=np.array(discriminator_gene_loss).mean(axis=0)
    
    d_loss = 0.5 * (discriminator_true_loss[0]+discriminator_gene_loss[0])
    g_loss=np.array(generator_loss).mean(axis=0)
    
    logs['d_loss'].append(d_loss)
    logs['d_true_loss'].append(discriminator_true_loss[0])
    logs['d_gene_loss'].append(discriminator_gene_loss[0])
    logs['g_loss'].append(g_loss)
    
    
    # Test discriminator on test data from Cifat10
    discriminator_true_val_loss=[]
    discriminator_gene_val_loss=[]
    for k in tqdm(range(int(X_test.shape[0] // BATCH_SIZE )),desc=('Epoch (test) #%d' % epoch), leave=False):
        imgs=X_test[k*BATCH_SIZE : (k+1)*BATCH_SIZE]
        d_val_loss = discriminator.test_on_batch(imgs, np.zeros((BATCH_SIZE, 1)))
        discriminator_true_val_loss.append(d_val_loss)
        
        noise_data = generate_noise(BATCH_SIZE, 100)
        generated_images = generator.predict(noise_data)
        d_gene_loss = discriminator.test_on_batch(generated_images, np.ones((BATCH_SIZE, 1)))
        discriminator_gene_val_loss.append(d_gene_loss)
    discriminator_true_val_loss=np.array(discriminator_true_val_loss).mean(axis=0)
    discriminator_gene_val_loss=np.array(discriminator_gene_val_loss).mean(axis=0)
    d_true_val_loss, d_true_val_acc  = discriminator_true_val_loss
    d_gene_val_loss, d_gene_val_acc  = discriminator_gene_val_loss
    d_val_loss=0.5*(d_true_val_loss+d_gene_val_loss)
    
    logs_val['d_val_loss'].append(d_val_loss)
    logs_val['d_true_val_loss'].append(d_true_val_loss)
    logs_val['d_gene_val_loss'].append(d_gene_val_loss)
    logs_val['d_true_val_acc'].append(d_true_val_acc)
    logs_val['d_gene_val_acc'].append(d_gene_val_acc)
    
    
    print ("%d [D loss: %f ] [G loss: %f]" % (epoch, d_loss, g_loss))
    print ("%d [D val_loss: %f, val_acc.real: %.2f%%, val_acc.gen: %.2f%%]" % (epoch, d_val_loss, 100*d_true_val_acc, 100*d_gene_val_acc))
    
    row_dict = OrderedDict({'epoch': list(range(initial_epoch,epoch+1))})
    row_dict.update((key, logs[key]) for key in logs.keys())
    row_dict.update((key, logs_val[key]) for key in logs_val.keys())
    df=pd.DataFrame.from_dict(row_dict)
    
    df.to_csv(os.path.join(SAVE_DIR,'logs.csv'),index=False)
    
    if epoch%SAVE_INTERVAL==0 or epoch==1:
        show_imgs(epoch,SAVE_DIR)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
discriminator.save_weights(os.path.join(SAVE_DIR,'discriminator.h5'))
gan.save_weights(os.path.join('gan.h5'))
generator.save_weights(os.path.join('generator.h5'))